# Dynamic Programming
- Similar to divide and conquer, dynamic programming solve a problem by compining solotions of subproblems. 
- However, in divide and conquer each subproblem is disjoint from the other, while in dynamic programming the subproblems overlap.
- Dynamic programming solves each subproblems, stores it and use it when it appears again in bigger-sized subproblems
- DP applies often to oprimization problems
- those problem has many solutions, but we want to find a solution with the optimal value.  
  
To develop a dynamic-protgramming algorithm we follow the four steps:
- Characterize the structure of an optimal solution, optimal substructure-> optimal solution incorporate optimal solutions to related subproblems
- Recursively define the value of an optimal solution
- compute the value of an optimal solution, typically in a bottom-up fashion
- construct an optimal solution from computed information.

Consider the cut-rod problem: Given a rod of length $L$ find a sequeance of cuts from the avaliable cuts $\{l_1,l_2,\cdots, l_n\}$ corresponding to prices $\{p_1,p_2,\cdots,p_n\}$ such that the total price is maximized:
$$ V = \max_{\{n_i\}}\{\sum\limits_{i=1}^{n} n_i\cdot p_i \} \,\,\, , \text{such that} \,\,\,  L = \sum\limits_{i=1}^{n}n_i\cdot l_i +w \,\,\, \text{with} \,\,\, w>0 \,\, \text{and} \,\, n_i\in \mathbb{N}$$

- Naive Recurnce: 
  mavalue(L, cuts, prices): 
   - if $L=0$: return $0$
   - if $L<0$: return $-\infin$
   - max_val = 0
   - for $l_i\in$ cuts:
      - q = max(q, maxvalue($L-l_i$, cuts, prices) )
   - return max_val  
   
However, we solve the same sub-problem every time it appears and the time complexity is $\mathcal{O}(2^L)$
- Top-Down Approach and Memoization:  
every time we solve a sub-problem store its optimal value on an array or a hash-table and if the subproblem appears again just recover its solution.  
We can also store the cut that corresponds to the optimal value, if we want also to find the optimal cut sequence as well as the optimal value
- Bottom-UP Approach:  
when the solution to a subproblem depends only on solving smaller-sized subproblems then we can solve the subproblems in size-order from smallest to the original.
- the time complexity for both top-down and bottom-up for the rod-cutting problem is $\mathcal{O}(L^2)$, but they use extra space $\mathcal{O}(2L)$

In [25]:
def maxvalue(l, sizes, prices):
    global t
    t+=1
    
    if l==0:
        return 0 
    if l<0:
        return float('-inf')
    
    return max([0]+[maxvalue(l-li, sizes, prices)+pi for (li,pi) in zip(sizes,prices)])

In [62]:
L = 10
sizes =  [ 1, 2, 3, 4]
prices = [ 1, 5, 8, 9]
t = 0
print(maxvalue(L,sizes, prices), t)

26 1729


In [59]:
def maxvalue_top_down(l: int, cuts: list, prices: list):
    # initialize the memoized list for optimal values and optimal cut for every possible length
    optimal_values = [float('-inf')]*(l+1)
    optimal_cuts = [float('-inf')]*(l+1)

    # set the value and cut for a length 0 to 0
    optimal_values[0] = optimal_cuts[0] = 0

    # call the recurrence
    optimal_val = _maxvalue_top_down(l, optimal_values, optimal_cuts, cuts, prices)

    # print the optimal cuts
    path = [l]
    length = l 
    while optimal_cuts[length]>0:
        length -= optimal_cuts[length]
        path.append(length)
    print(f'The optimal cuts are {','.join(str(length) for length in path)} and optimal value {optimal_values[-1]}')

    return optimal_val, optimal_values, optimal_cuts

def _maxvalue_top_down(l: int, optimal_values: list, optimal_cuts: list, cuts: list, prices: list):
    # check if there is no cut -> waste
    if l<0: 
        return float('-inf')
    
    # check if sub-problem for length l is already solved, includes the l=0 case
    if optimal_values[l]>=0:
        return optimal_values[l]

    
    # find the max_value and the corresponding cut, consider also the 0 value in case of all cuts leading to a waste
    # in case of a tie for the optimal value of two cuts, keep the bigger cut
    max_val, max_cut = max([(0,0)]+[(_maxvalue_top_down(l-li, optimal_values, optimal_cuts, cuts, prices)+pi, li) for (li,pi) in zip(cuts, prices)])

    # memoize the optimal solution
    optimal_values[l] = max_val
    optimal_cuts[l] = max_cut

    return max_val

In [60]:
sizes =  [ 1, 3, 5, 10, 30, 50, 75]
prices = [ 0.1, 0.2, 0.4, 0.9, 3.1, 5.1, 8.2]
for L in (30,50,100,130, 300):
    maxvalue_top_down(L, sizes, prices)

The optimal cuts are 30,0 and optimal value 3.1
The optimal cuts are 50,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,0 and optimal value 5.1000000000000005
The optimal cuts are 100,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,0 and optimal value 10.7
The optimal cuts are 130,55,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,0 and optimal value 13.8
The optimal cuts are 300,225,150,75,0 and optimal value 32.8


In [49]:
def maxvalue_bottom_up(l: int, cuts: list, prices: list):
    # initialize the memoized list for optimal values and optimal cut for every possible length
    optimal_values = [0]*(l+1)
    optimal_cuts = [0]*(l+1)
    
    # consider all sizes for the rod, staring at 1, 2, ..., l
    for l_size in range(1, l+1):
        # find the optimal value and the corresponding optimal cut for the rod of size l_size, add also the 0 value in case of waste
        max_val, max_cut = max([(0,0)] + [(optimal_values[l_size-li] + pi, li) for (li, pi) in zip(cuts, prices) if l_size-li>=0] )
        
        # memoize the optimal solution
        optimal_values[l_size] = max_val
        optimal_cuts[l_size] = max_cut

    # print the optimal cuts
    path = [l]
    length = l 
    while optimal_cuts[length]>0:
        length -= optimal_cuts[length]
        path.append(length)
    print(f'The optimal cuts are {','.join(str(length) for length in path)} and optimal value {optimal_values[-1]}')

    return  optimal_values, optimal_cuts

In [61]:
sizes =  [ 1, 3, 5, 10, 30, 50, 75]
prices = [ 0.1, 0.2, 0.4, 0.9, 3.1, 5.1, 8.2]
for L in (30,50,100,130, 300):
    maxvalue_bottom_up(L, sizes, prices)

The optimal cuts are 30,0 and optimal value 3.1
The optimal cuts are 50,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,0 and optimal value 5.1000000000000005
The optimal cuts are 100,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,0 and optimal value 10.7
The optimal cuts are 130,55,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,0 and optimal value 13.8
The optimal cuts are 300,225,150,75,0 and optimal value 32.8


### Different Dynamic Approach
For a given rod-length $L$ Instead of finding the maximum value of all possible cuts, find the maximum value of all possible number of cuts of length $l_1$, i.e $n_1 = 0,1,\dots, \lfloor L/l_1 \rfloor$, which will depend on the maximum value of all possible number of cuts of length $l_2$ for the rod-length of $L-n_1\cdot l_1$, $n_2 = 0,1, \cdots, \lfloor L/l_2 \rfloor$ and so on.  
So consider the following notation:
 - For a rod of length $L$, 
 - avaliable cuts $c= \{l_1,l_2,\cdots,l_k\}$ 
 - and prices $p=\{p_1,p_2,\cdots,p_k\}$, 
 - let $i=1,\cdots, k$ be the index numbering the avaliable cuts.  
 - denote $n_i = 0,1,\dots, k_i \equiv \lfloor L/l_i \rfloor $  
 - such that $L = \sum\limits_{i=1}^{k}n_i\cdot l_i + w$.   
Simalar to previously we can use a naive recursive approacn, or a bottom-up approach

In [15]:
def max_value_n(l: int, i: int, cuts: list, prices: list):
    if l<0:
        return float('-inf')
    if l==0:
        return 0
    if i>=len(cuts):
        return 0
    ki = int(l/cuts[i])
    if ki==0:
        return 0
    return max([0]+[max_value_n(l-ni*cuts[i] , i+1, cuts, prices) +ni*prices[i] for ni in range(ki+1)])

In [17]:
sizes =  [ 1, 3, 5, 10, 30, 50, 75]
prices = [ 0.1, 0.2, 0.4, 0.9, 3.1, 5.1, 8.2]
for L in (30,50,100, 130, 300):
    print(f'max value for rod of length {L} is {max_value_n(L, 0,sizes, prices)}')

max value for rod of length 30 is 3.1
max value for rod of length 50 is 5.1
max value for rod of length 100 is 10.7
max value for rod of length 130 is 13.799999999999999
max value for rod of length 300 is 32.8


In [46]:
def bottom_up_n(l: int, cuts: list, prices: list):
    k = len(sizes)
    values = []
    numbers = []
    for i in range(l+1):
        values.append([0]*(k+1))
        numbers.append([-1]*(k))
        
    for l in range(l+1):
        for i in range(k-1,-1,-1):
            ki = int(l/cuts[i])
            max_val, max_val_ni = max([(0, -1)]+[(values[l-ni*cuts[i]][i+1] +ni*prices[i], ni) for ni in range(ki+1) ])
            values[l][i] = max_val
            numbers[l][i] = max_val_ni
    
    optimal_cut_numbers = []
    length = l
    for i in range(k):
        ni = numbers[length][i]
        optimal_cut_numbers.append(ni)
        length = length-ni*cuts[i]        
    
    print(optimal_cut_numbers)
    opt_cuts_stinrg = ', '.join([str(ni)+' cuts of size '+str(cuts[i]) for i,ni in enumerate(optimal_cut_numbers) if ni>0])
    print(f'the optimal number of cuts for rod-length of {l} are: {opt_cuts_stinrg} with optimal value {values[l][0]}')
    
    return values[l][0]

In [47]:
sizes =  [ 1, 3, 5, 10, 30, 50, 75]
prices = [ 0.1, 0.2, 0.4, 0.9, 3.1, 5.1, 8.2]
for L in (30,50,100, 130, 300):
    bottom_up_n(L, sizes, prices)

[0, 0, 0, 0, 1, 0, 0]
the optimal number of cuts for rod-length of 30 are: 1 cuts of size 30 with optimal value 3.1
[20, 0, 0, 0, 1, 0, 0]
the optimal number of cuts for rod-length of 50 are: 20 cuts of size 1, 1 cuts of size 30 with optimal value 5.1
[25, 0, 0, 0, 0, 0, 1]
the optimal number of cuts for rod-length of 100 are: 25 cuts of size 1, 1 cuts of size 75 with optimal value 10.7
[25, 0, 0, 0, 1, 0, 1]
the optimal number of cuts for rod-length of 130 are: 25 cuts of size 1, 1 cuts of size 30, 1 cuts of size 75 with optimal value 13.799999999999999
[0, 0, 0, 0, 0, 0, 4]
the optimal number of cuts for rod-length of 300 are: 4 cuts of size 75 with optimal value 32.8
